#### Import the needed libraries

In [52]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

print('Libraries imported.')

Libraries imported.


#### Download the Data from the website and convert it to a dataframe

In [53]:
list = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = pd.DataFrame(list[0])
df.head() 

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Check the shape of the dataframe

In [54]:
df.shape

(287, 3)

#### Find all boroughs which are not assigned and replace them with NaN

In [55]:
df["Borough"] = df["Borough"].replace(to_replace="Not assigned",value = np.nan)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Get the number of Nulls, i.e. not assigned boroughs

In [56]:
df.isnull().sum()

Postcode          0
Borough          77
Neighbourhood     0
dtype: int64

####  Drop all rows with not assigned boroughs and reset the index

In [57]:
df = df.dropna(axis=0)
df = df.reset_index(drop=True)

#### Rename Postcode to Postalcode

In [58]:
df.columns =['PostalCode', 'Borough', 'Neighbourhood']

In [59]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### Check the shape of the dataframe again

In [60]:
df.shape

(210, 3)

#### Get postcodes from csv-File

In [61]:
postal_codes = pd.read_csv("https://cocl.us/Geospatial_data")
postal_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Rename postcode to PostalCode

In [62]:
postal_codes.columns = ['PostalCode', 'Latitude', 'Longitude']
postal_codes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge df mit postal_codes

In [63]:
df_new = pd.merge(left = df, right = postal_codes, on = "PostalCode")
df_new.head(20)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188


#### Install and import folium to create a map of Toronto

In [64]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print("Folium is installed and imported")

Folium is installed and imported


#### Install and import Nominatim from geocoders to get Latitude and Longitude from Toronto

In [65]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
print("Nominatim is installed and imported")

Nominatim is installed and imported


#### Get the Latitude and Longitude from Toronto

In [66]:
address = 'Toronto'

In [67]:
geolocator = Nominatim(user_agent="toronto_explorer") # name of the user_agent: toronto_explorer
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


####  Import and install folium to create a map of Toronto and its neighborhoods

In [68]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print("Folium ist installed and imported")

Folium ist installed and imported


#### Rename df_new in neighborhoods

In [69]:
neighborhoods = df_new
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


#### Create a map of Toronto and its neighborhoods

In [70]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
   
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5, 
        popup=label, 
        color='blue', 
        fill=True,    
        fill_color='#3186cc', 
        fill_opacity=0.7, 
        parse_html=False).add_to(map_toronto) 



map_toronto

#### Start utilizing the Foursquare API to explore the neighborhoods and segment them

##### Define Foursquare Credentials and Version

In [71]:
# The code was removed by Watson Studio for sharing.

#### Let's check the Boroughs in Toronto

In [72]:
neighborhoods["Borough"].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

## Let's explore the neighborhoods of 'Downtown Toronto'

In [73]:
# create therefore a subset of the boroughs of toronto: 'Downtown Toronto'
downtown_toronto = neighborhoods[(neighborhoods['Borough']=='Downtown Toronto')].drop("PostalCode", axis=1).reset_index(drop=True)
downtown_toronto.columns =['Borough', 'Neighborhood', 'Latitude', 'Longitude']
downtown_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,Downtown Toronto,Ryerson,43.657162,-79.378937
3,Downtown Toronto,Garden District,43.657162,-79.378937
4,Downtown Toronto,St. James Town,43.651494,-79.375418


#### Let's create a function to get the top 100 venues of Downtown Toronto

In [76]:
# search for max. 100 venues
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500): 
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        import requests
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Code to run the above function on each neighborhood and create a new dataframe called *downtown_toronto_venues*.

In [77]:
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto['Neighborhood'],
                                   latitudes=downtown_toronto['Latitude'],
                                   longitudes=downtown_toronto['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


#### Let's check the size of the resulting dataframe

In [78]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(2455, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


### Let's examine the number of distinct venues

In [79]:
downtown_toronto_venues["Venue"].nunique()

743

There are 2,455 venues in Downtown Toronto, but only 743 distinct venues because there is a lot of overlap.

#### Let's check how many venues were returned for each neighborhood

In [80]:
downtown_toronto_venues.groupby('Neighborhood')['Venue'].count().sort_values(ascending = False)

Neighborhood
St. James Town                     146
Victoria Hotel                     100
Richmond                           100
Commerce Court                     100
Design Exchange                    100
First Canadian Place               100
Garden District                    100
Harbourfront East                  100
King                               100
Adelaide                           100
Toronto Dominion Centre            100
Union Station                      100
Underground city                   100
Toronto Islands                    100
Ryerson                            100
Stn A PO Boxes 25 The Esplanade     95
Church and Wellesley                86
Grange Park                         85
Chinatown                           85
Kensington Market                   85
Central Bay Street                  78
Berczy Park                         56
Cabbagetown                         46
Harbourfront                        46
Queen's Park                        41
University o

#### Let's find out how many unique categories can be created from all the returned venues

In [81]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


#### Set option to display max columns and max rows

In [82]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

## Let's analyze each neighborhood 

In [83]:
# one hot encoding of the 'Venue Category'
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="").drop("Neighborhood", axis=1)

# add neighborhood column back to dataframe
neighborhood = downtown_toronto_venues['Neighborhood']
downtown_toronto_onehot = pd.concat([neighborhood,downtown_toronto_onehot], axis = 1)
downtown_toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Examine the shape of the dataframe

In [84]:
downtown_toronto_onehot.shape

(2455, 206)

#### Group the dataframe by neighbourhoods and get their average frequency of the categories

In [85]:
downtown_toronto_grouped =  downtown_toronto_onehot.groupby(by = "Neighborhood").mean().reset_index()
downtown_toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.020000,0.0,0.0,0.010000,0.0,0.02,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.0400,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.01,0.0000,0.01,0.020000,0.0,0.0,0.01,0.02,0.01,0.000000,0.040000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.02,0.000000,0.070000,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.020000,0.000000,0.02,0.000000,0.0,0.010000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.01,0.0,0.000000,0.0,0.01,0.0,0.000000,0.000000,0.0,0.01,0.0,0.0,0.020000,0.0,0.000000,0.01,0.0,0.010000,0.01,0.000000,0.01,0.000000,0.02,0.01,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.0,0.0,0.0,0.01,0.0,0.000000,0.020000,0.000000,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.000000,0.0,0.01,0.000000,0.01,0.01,0.01,0.0,0.0,0.0,0.000000,0.010000,0.0,0.000000,0.000000,0.020000,0.000000,0.01,0.01,0.0,0.0,0.000000,0.01,0.01,0.0000,0.040000,0.0,0.0,0.01,0.01,0.000000,0.0,0.0000,0.0200

All frequencies in a row add up to 1, for example the second row

In [86]:
downtown_toronto_grouped.iloc[1,1:].to_frame().sum()

1    1.0
dtype: float64

#### Check the size of the new dataframe

In [87]:
downtown_toronto_grouped.shape 

(36, 206)

Downtown Toronto has 36 neighborhoods and 206 categories of venues

#### Let's print each neighborhood along with the top 5 most common venues and their frequency

In [88]:
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("---"+hood+"---")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Adelaide---
             Venue  Frequency
0      Coffee Shop       0.07
1             Café       0.04
2              Bar       0.04
3  Thai Restaurant       0.04
4       Restaurant       0.04


---Bathurst Quay---
             Venue  Frequency
0  Airport Service       0.19
1   Airport Lounge       0.12
2      Coffee Shop       0.06
3         Boutique       0.06
4              Bar       0.06


---Berczy Park---
            Venue  Frequency
0     Coffee Shop       0.09
1        Beer Bar       0.04
2  Farmers Market       0.04
3            Café       0.04
4     Cheese Shop       0.04


---CN Tower---
             Venue  Frequency
0  Airport Service       0.19
1   Airport Lounge       0.12
2      Coffee Shop       0.06
3         Boutique       0.06
4              Bar       0.06


---Cabbagetown---
         Venue  Frequency
0  Coffee Shop       0.07
1         Park       0.04
2       Market       0.04
3       Bakery       0.04
4         Café       0.04


---Central Bay Street---
         

#### Let's create a dataframe

First, it's needed to write a function to sort the venues in descending order.

In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =   downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Thai Restaurant,Bar,Sushi Restaurant,Breakfast Spot,Hotel,Steakhouse,Cosmetics Shop
1,Bathurst Quay,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
2,Berczy Park,Coffee Shop,Beer Bar,Seafood Restaurant,Cocktail Bar,Restaurant,Bakery,Cheese Shop,Farmers Market,Café,Bistro
3,CN Tower,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
4,Cabbagetown,Coffee Shop,Park,Market,Restaurant,Pub,Bakery,Pizza Place,Café,Italian Restaurant,Chinese Restaurant


## Now, let's cluster Downtown Toronto

#### Use KMeans for Clustering from sklearn

##### Import KMeans-Library

In [91]:
from sklearn.cluster import KMeans
print("KMeans is imported")

KMeans is imported


#### Cluster the neighborhoods into 5 cluster

In [92]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', axis = 1)

# train KMeans
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for the first 10 rows in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 1, 0, 0, 2, 4, 0, 0], dtype=int32)

#### Let's join the dataframe of downtown toronto with the dataframe of the top 10 venues of each neighborhood

In [93]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = downtown_toronto

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Electronics Store,Distribution Center
1,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Park,Burger Joint,Beer Bar,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Chinese Restaurant,College Auditorium
2,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
3,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
4,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Park,Hotel,Diner,Clothing Store,Breakfast Spot


#### Let's visualize these clusters on the map of toronto

Import Matplotlib and relevant plotting modules

In [94]:
import matplotlib.cm as cm
import matplotlib.colors as colors
print("Matplotlib is imported")

Matplotlib is imported


In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1.0).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Define columns

In [96]:
columns = ['Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels',
       '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue']

#### Cluster 1 (Coffee-Shop-Cluster)

In [97]:
cluster_1 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, columns]
cluster_1

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Electronics Store,Distribution Center
1,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Park,Burger Joint,Beer Bar,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Chinese Restaurant,College Auditorium
2,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
3,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
4,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Park,Hotel,Diner,Clothing Store,Breakfast Spot
5,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Beer Bar,Seafood Restaurant,Cocktail Bar,Restaurant,Bakery,Cheese Shop,Farmers Market,Café,Bistro
6,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Burger Joint,Sandwich Place,Japanese Restaurant,Ice Cream Shop,Department Store,Salad Place,Bar,Bubble Tea Shop
8,Adelaide,43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Bar,Sushi Restaurant,Breakfast Spot,Hotel,Steakhouse,Cosmetics Shop
9,King,43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Bar,Sushi Restaurant,Breakfast Spot,Hotel,Steakhouse,Cosmetics Shop
10,Richmond,43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Bar,Sushi Restaurant,Breakfast Spot,Hotel,Steakhouse,Cosmetics Shop


In [98]:
len(cluster_1)

23

This cluster consists of 23 neighborhoods. Its 1st Most Common Venue in all neighborhoods is Coffee Shop. 
This is therefore the Coffee-Shop-Cluster.

#### Cluster 2 (Airport-Cluster)

In [99]:
cluster_2 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, columns]
cluster_2

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,CN Tower,43.628947,-79.39442,1,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
24,Bathurst Quay,43.628947,-79.39442,1,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
25,Island airport,43.628947,-79.39442,1,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
26,Harbourfront West,43.628947,-79.39442,1,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
27,King and Spadina,43.628947,-79.39442,1,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
28,Railway Lands,43.628947,-79.39442,1,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport
29,South Niagara,43.628947,-79.39442,1,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport


In [100]:
len(cluster_2)

7

Cluster 2 consists of 7 clusters. The 1st and 2nd Most Common Venues in all neighborhoods are 'Airport Service' and 
'Airport Lounge'. Therefore this cluster is called Airport-Cluster.

#### Cluster 3 (Gastronomy-Cluster)

In [101]:
cluster_3 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, columns]
cluster_3

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Harbord,43.662696,-79.400049,2,Café,Bakery,Italian Restaurant,Japanese Restaurant,Bookstore,Restaurant,Bar,Chinese Restaurant,Dessert Shop,Coffee Shop
19,University of Toronto,43.662696,-79.400049,2,Café,Bakery,Italian Restaurant,Japanese Restaurant,Bookstore,Restaurant,Bar,Chinese Restaurant,Dessert Shop,Coffee Shop
20,Chinatown,43.653206,-79.400049,2,Bar,Vietnamese Restaurant,Bakery,Café,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Dessert Shop
21,Grange Park,43.653206,-79.400049,2,Bar,Vietnamese Restaurant,Bakery,Café,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Dessert Shop
22,Kensington Market,43.653206,-79.400049,2,Bar,Vietnamese Restaurant,Bakery,Café,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Dessert Shop


Cluster 3 consists of 5 neighbourhoods. Most of their venues revolve around gastronomy.
This is therefore the Gastronomy-Cluster.

#### Cluster 4 (Nature-Cluster)

In [102]:
cluster_4 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, columns]
cluster_4

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


Cluster 4 consists of only one neighbourhood. Its venues are mainly located in the open nature. Therefore it is the Nature-Cluster.

#### Cluster 5 (Miscellaneous-Cluster)

In [103]:
cluster_5 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, columns]
cluster_5

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Gas Station,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Candy Store


Cluster 5 consists of only one neighbourhood,too. Its venues are completely different. Therefore it is
the Miscellaneous-Cluster.